In [5]:
from snowflake.snowpark import Session

In [26]:
import sklearn
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore')
import configparser

# Code to establish connection and read data from Snowflake

In [7]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [8]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [9]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [10]:
application_train_sf  = session.table("CRA_APPLICATION_TRAIN_DETAILS")
application_test_sf  = session.table("CRA_APPLICATION_TEST_DETAILS")
bureau_sf  = session.table("CRA_BUREAU_DETAILS")
bureau_balance_sf  = session.table("CRA_BUREAU_BALANCE_DETAILS")
credit_card_balance_sf  = session.table("CRA_CREDIT_CARD_BALANCE_DETAILS")
installments_payments_sf  = session.table("CRA_INSTALLMENTS_PAYMENTS_DETAILS")
previous_application_sf  = session.table("CRA_PREVIOUS_APPLICATION_DETAILS")
POS_CASH_balance_sf  = session.table("CRA_POS_CASH_BALANCE_DETAILS")

# Convert Snowflake data into Pandas dataframes

In [11]:
application_train = application_train_sf.to_pandas()

# Glimpse of Rows and Columns of each dataset

In [12]:
print("application_train -  rows:",application_train.shape[0]," columns:", application_train.shape[1])

application_train -  rows: 307511  columns: 124


# Sample Data of each dataset

In [13]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CREATED_BY,CREATED_AT
0,307660,0,Cash loans,M,Y,Y,0,180000.0,1288350.0,41692.5,...,0,0,0.0,0.0,0.0,0.0,1.0,2.0,manishh.singh@fosfor.com,2024-05-03 06:30:22
1,307662,0,Cash loans,F,N,Y,1,90000.0,93829.5,9981.0,...,0,0,0.0,0.0,0.0,0.0,0.0,4.0,manishh.singh@fosfor.com,2024-05-03 06:30:22
2,307663,0,Cash loans,F,N,Y,0,360000.0,2517300.0,66402.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,manishh.singh@fosfor.com,2024-05-03 06:30:22
3,307664,0,Revolving loans,F,N,N,0,90000.0,270000.0,13500.0,...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,manishh.singh@fosfor.com,2024-05-03 06:30:22
4,307665,0,Revolving loans,M,Y,Y,0,225000.0,540000.0,27000.0,...,0,0,0.0,0.0,0.0,0.0,2.0,7.0,manishh.singh@fosfor.com,2024-05-03 06:30:22


In [16]:
application_train.drop(['CREATED_BY','CREATED_AT'], inplace=True, axis=1)

In [19]:
print("application_train -  rows:",application_train.shape[0]," columns:", application_train.shape[1])

application_train -  rows: 307511  columns: 122


# Stratified Sampling (ratio=0.1)

In [17]:
application_sample1 = application_train.loc[application_train.TARGET==1].sample(frac=0.1, replace=False)
print('label 1 sample size:', str(application_sample1.shape[0]))
application_sample0 = application_train.loc[application_train.TARGET==0].sample(frac=0.1, replace=False)
print('label 0 sample size:', str(application_sample0.shape[0]))
application = pd.concat([application_sample1, application_sample0], axis=0).sort_values('SK_ID_CURR')

label 1 sample size: 2482
label 0 sample size: 28269


## Impute missing value

In [18]:
categorical_list = []
numerical_list = []
for i in application.columns.tolist():
    if application[i].dtype=='object':
        categorical_list.append(i)
    else:
        numerical_list.append(i)
print('Number of categorical features:', str(len(categorical_list)))
print('Number of numerical features:', str(len(numerical_list)))

Number of categorical features: 16
Number of numerical features: 106


In [24]:
from sklearn.impute import SimpleImputer as Imputer
application[numerical_list] = Imputer(strategy='median').fit_transform(application[numerical_list])

In [28]:
#del application_train; gc.collect()
application = pd.get_dummies(application, drop_first=True)
print(application.shape)

(30751, 234)


### Feature matrix and target

In [29]:
X = application.drop(['SK_ID_CURR', 'TARGET'], axis=1)
y = application.TARGET
feature_name = X.columns.tolist()

### 1. Filer method using Pearson Correlation
#### Normalization: no
#### Impute missing values: yes

In [30]:
def cor_selector(X, y):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-100:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

In [31]:
cor_support, cor_feature = cor_selector(X, y)
print(str(len(cor_feature)), 'selected features')

100 selected features


In [35]:
print (cor_feature)

['ORGANIZATION_TYPE_Transport: type 3', 'WALLSMATERIAL_MODE_Stone, brick', 'FLAG_DOCUMENT_13', 'FONDKAPREMONT_MODE_reg oper spec account', 'FONDKAPREMONT_MODE_reg oper account', 'COMMONAREA_MEDI', 'BASEMENTAREA_MODE', 'COMMONAREA_AVG', 'ORGANIZATION_TYPE_Trade: type 7', 'AMT_ANNUITY', 'ORGANIZATION_TYPE_Construction', 'OWN_CAR_AGE', 'OCCUPATION_TYPE_Managers', 'BASEMENTAREA_MEDI', 'OCCUPATION_TYPE_Core staff', 'BASEMENTAREA_AVG', 'ORGANIZATION_TYPE_Transport: type 4', 'ORGANIZATION_TYPE_Industry: type 6', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE_Cleaning', 'ORGANIZATION_TYPE_Government', 'NONLIVINGAREA_MODE', 'NONLIVINGAREA_MEDI', 'OCCUPATION_TYPE_Low-skill Laborers', 'LIVINGAPARTMENTS_MODE', 'ORGANIZATION_TYPE_Military', 'LIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'APARTMENTS_MODE', 'ORGANIZATION_TYPE_Restaurant', 'FONDKAPREMONT_MODE_org spec account', 'CNT_CHILDREN', 'OCCUPATION_TYPE_Sales staff', 'NAME_INCOME_T

### 2. Chi-Square 
#### Normalization: MinMaxScaler (values should be bigger than 0)
#### Impute missing values: yes

In [36]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=100)
chi_selector.fit(X_norm, y)

SelectKBest(k=100, score_func=<function chi2 at 0x7faded115a60>)

In [37]:
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

100 selected features


In [38]:
print (chi_feature)

['AMT_CREDIT', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'ELEVATORS_AVG', 'FLOORSMAX_AVG', 'LIVINGAREA_AVG', 'ELEVATORS_MODE', 'FLOORSMAX_MODE', 'ELEVATORS_MEDI', 'FLOORSMAX_MEDI', 'LIVINGAREA_MEDI', 'TOTALAREA_MODE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_21', 'NAME_CONTRACT_TYPE_Revolving loans', 'CODE_GENDER_M', 'FLAG_OWN_CAR_Y', 'NAME_TYPE_SUITE_Nan values', 'NAME_TYPE_SUITE_Other_A', 'NAME_INCOME_TYPE_Pensioner', 'NAME_INCOME_TYPE_State servant', 'NAME_INCOME_TYPE_Unemployed', 'NAME_I

### 3. Wrapper method using RFE (Recursive Feature Elimination)
#### Normalization: depend on the used model; yes for LR
#### Impute missing values: depend on the used model; yes for LR

In [39]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=100, step=10, verbose=5)
rfe_selector.fit(X_norm, y)

Fitting estimator with 232 features.
Fitting estimator with 222 features.
Fitting estimator with 212 features.
Fitting estimator with 202 features.
Fitting estimator with 192 features.
Fitting estimator with 182 features.
Fitting estimator with 172 features.
Fitting estimator with 162 features.
Fitting estimator with 152 features.
Fitting estimator with 142 features.
Fitting estimator with 132 features.
Fitting estimator with 122 features.
Fitting estimator with 112 features.
Fitting estimator with 102 features.


RFE(estimator=LogisticRegression(), n_features_to_select=100, step=10,
    verbose=5)

In [40]:
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

100 selected features


In [41]:
print (rfe_feature)

['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_CONT_MOBILE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_AVG', 'LANDAREA_AVG', 'LIVINGAREA_AVG', 'APARTMENTS_MODE', 'COMMONAREA_MODE', 'LANDAREA_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'APARTMENTS_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'ELEVATORS_MEDI', 'FLOORSMAX_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'TOTALAREA_MODE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUME

### 4. Embedded Method using Select From Model
#### Normalization: Yes
#### Impute missing values: Yes

In [45]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(estimator=LogisticRegression(penalty="l2"))
embeded_lr_selector.fit(X_norm, y)

SelectFromModel(estimator=LogisticRegression())

In [46]:
embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

74 selected features


In [47]:
print (embeded_lr_feature)

['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_CONT_MOBILE', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_AVG', 'LANDAREA_AVG', 'LIVINGAREA_AVG', 'APARTMENTS_MODE', 'LIVINGAPARTMENTS_MODE', 'APARTMENTS_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'FLOORSMAX_MEDI', 'LANDAREA_MEDI', 'LIVINGAREA_MEDI', 'TOTALAREA_MODE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'CODE_GENDER_M', 'FLAG_OWN_CAR_Y', 'NAME_TYPE_SUITE_Group of people', 'NAME_TYPE_SUITE_N

### 5 Random Forest Feature Importance
#### Normalization: No
#### Impute missing values: Yes

In [48]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), threshold='1.25*median')
embeded_rf_selector.fit(X, y)

SelectFromModel(estimator=RandomForestClassifier(), threshold='1.25*median')

In [49]:
embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

101 selected features


In [50]:
print (embeded_rf_feature)

['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIV

## Summary

In [53]:
feature_selection_df.columns

Index(['Feature', 'Pearson', 'Chi-2', 'RFE', 'Logistics', 'Random Forest'], dtype='object')

In [62]:
pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    'Random Forest':embeded_rf_support})

feature_selection_df_temp = feature_selection_df[['Pearson', 'Chi-2', 'RFE', 'Logistics', 'Random Forest']]
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df_temp, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(100)

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,Total
1,TOTALAREA_MODE,True,True,True,True,True,5
2,REGION_RATING_CLIENT_W_CITY,True,True,True,True,True,5
3,REGION_RATING_CLIENT,True,True,True,True,True,5
4,LIVINGAREA_MEDI,True,True,True,True,True,5
5,LIVINGAREA_AVG,True,True,True,True,True,5
6,FLOORSMAX_MEDI,True,True,True,True,True,5
7,FLAG_OWN_CAR_Y,True,True,True,True,True,5
8,EXT_SOURCE_3,True,True,True,True,True,5
9,EXT_SOURCE_2,True,True,True,True,True,5
10,EXT_SOURCE_1,True,True,True,True,True,5
